<a href="https://colab.research.google.com/github/adarsh182005/Logistics-Optimization-Project/blob/main/logistics_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📦 Order Fulfilment & Logistics Optimization Pipeline

## 🚀 Project Overview
An automated data engineering pipeline designed to optimize logistics operations. This tool ingests raw, unstructured delivery logs (CSV) and transforms them into actionable business intelligence.

## 🔑 Key Capabilities
* **Dynamic Sessionization:** Algorithmically detects "Order Sessions" from continuous GPS/timestamp streams using time-gap thresholding.
* **Transit Time Analysis:** Computes accurate delivery durations, filtering out outliers using Median statistics.
* **Courier Performance:** Aggregates metrics to evaluate agent efficiency and speed.
* **Forecasting Engine:** Utilizes Exponential Smoothing (Holt-Winters) to predict future transit trends.
* **Automated Reporting:** Exports clean, structured datasets for dashboarding tools (Power BI/Tableau).

## ⚙️ How It Works (Standard Operating Procedure)
1.  **Ingest:** The script accepts any raw logistics CSV.
2.  **Process:** It automatically detects time and ID columns, handling missing data gracefully.
3.  **Analyze:** Runs statistical models to extract KPIs.
4.  **Output:** Generates `order_times.csv`, `courier_kpis.csv`, and `monthly_trends.csv`.

## 🛠 Usage
1.  Open the notebook in **Google Colab** or **Jupyter**.
2.  Run all cells.
3.  Upload your dataset when prompted (or place it in the root directory).

In [2]:

# Install libraries (uncomment in fresh Colab)
# !pip install pandas plotly statsmodels

import pandas as pd, numpy as np, os
import plotly.express as px
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

print("Libraries ready")


Libraries ready


In [10]:

# ✅ STEP 1: Upload your dataset
from google.colab import files
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

print("Loaded:", file_name)
print("Shape:", df.shape)
display(df.head())


Saving orders_1000.csv to orders_1000.csv
Loaded: orders_1000.csv
Shape: (1000, 6)


,order_uid,accept_time,delivery_time,transit_hours,courier_id,month
0,1001,2024-11-23 09:01:47,2024-11-23 14:32:11,5.51,C008,Nov
1,1002,2024-04-24 10:47:06,2024-04-24 21:12:47,10.43,C003,Apr
2,1003,2024-10-29 19:02:01,2024-10-29 21:14:21,2.21,C007,Oct
3,1004,2024-04-29 22:38:01,2024-04-30 09:21:02,10.72,C007,Apr
4,1005,2024-11-28 19:14:28,2024-11-29 06:28:06,11.23,C009,Nov


In [11]:
# Check the raw format of the Delivery Time column
print("Sample Raw Delivery Times:")
print(df[delivery_col].head(5))

# Check if they are datetime objects or just dates
print("\nData Type:")
print(df[delivery_col].dtype)

Sample Raw Delivery Times:
0    2024-11-23 14:32:11
1    2024-04-24 21:12:47
2    2024-10-29 21:14:21
3    2024-04-30 09:21:02
4    2024-11-29 06:28:06
Name: delivery_time, dtype: object

Data Type:
object


In [12]:

# ✅ STEP 2: Detect & parse time column
time_candidates = [c for c in df.columns if "time" in c.lower() or "date" in c.lower()]
print("Detected time columns:", time_candidates)

time_col = time_candidates[0]
df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
df = df.sort_values(time_col)

print("Using time column:", time_col)


Detected time columns: ['accept_time', 'delivery_time']
Using time column: accept_time


In [13]:

# ✅ STEP 3: Detect courier / agent
id_candidates = [c for c in df.columns if "postman" in c.lower() or "courier" in c.lower() or "agent" in c.lower()]

if not id_candidates:
    df["courier_id"] = "C_" + (df.index // 30).astype(str)
    courier_col = "courier_id"
    print("Auto-created courier column:", courier_col)
else:
    courier_col = id_candidates[0]
    print("Using courier column:", courier_col)


Using courier column: courier_id


In [14]:

# ✅ STEP 4: Create Sessions (Orders)
df = df.sort_values([courier_col, time_col])
df["dt_min"] = df.groupby(courier_col)[time_col].diff().dt.total_seconds().div(60).fillna(0)

GAP_MIN = 60
df["new_session"] = (df["dt_min"] > GAP_MIN).astype(int)
df["order_id"] = df.groupby(courier_col)["new_session"].cumsum()
df["order_uid"] = df[courier_col].astype(str) + "_" + df["order_id"].astype(str)


In [15]:
# ✅ STEP 5: Order-Level Transit Calculation (Corrected & Robust)

# 1. Check if we have the column name variables (from Step 2)
if 'time_col' not in locals():
    print("⚠️ Error: 'time_col' variable not found. Please run Step 2 first.")
    time_col = 'order_time' # Fallback attempt

# 2. Identify the delivery column dynamically if needed
delivery_col = 'delivery_time'
if delivery_col not in df.columns:
    # Try to find a column containing "delivery"
    candidates = [c for c in df.columns if 'delivery' in c.lower()]
    if candidates:
        delivery_col = candidates[0]
        print(f"Using delivery column: {delivery_col}")

# 3. Ensure timestamps are in the correct datetime format
df[delivery_col] = pd.to_datetime(df[delivery_col], errors='coerce')
df[time_col] = pd.to_datetime(df[time_col], errors='coerce')

# 4. Select the specific columns we need using the variables
order_times = df[['order_uid', time_col, delivery_col]].copy()

# 5. Rename columns to standard names for calculation
order_times.rename(columns={time_col: 'accept_time', delivery_col: 'delivery_time'}, inplace=True)

# 6. Calculate Transit Hours (Delivery - Accept)
order_times["transit_hours"] = (order_times["delivery_time"] - order_times["accept_time"]).dt.total_seconds() / 3600

# 7. Remove invalid rows
order_times = order_times.dropna()

print(f"Calculated transit times for {len(order_times)} orders.")
display(order_times.head())

Calculated transit times for 1000 orders.


,order_uid,accept_time,delivery_time,transit_hours
726,C001_0,2024-01-26 21:58:17,2024-01-27 05:15:05,7.280000
750,C001_1,2024-01-30 18:42:20,2024-01-30 21:45:10,3.047222
698,C001_2,2024-02-17 16:38:39,2024-02-18 01:55:55,9.287778
816,C001_3,2024-02-28 09:48:42,2024-02-28 14:20:54,4.536667
857,C001_4,2024-02-28 18:16:34,2024-02-28 23:55:29,5.648611


In [16]:

# ✅ STEP 6: KPIs
total_orders = len(order_times)
avg_transit = round(order_times["transit_hours"].mean(), 2)
median_transit = round(order_times["transit_hours"].median(), 2)

print("Total Orders:", total_orders)
print("Avg Transit Hours:", avg_transit)
print("Median Transit Hours:", median_transit)


Total Orders: 1000
Avg Transit Hours: 6.41
Median Transit Hours: 6.47


In [17]:

# ✅ STEP 7: Courier Performance
mapping = df.groupby("order_uid")[courier_col].first().reset_index()
order_times = order_times.merge(mapping, on="order_uid")

courier_kpis = order_times.groupby(courier_col).agg(
    total_orders=("order_uid", "count"),
    avg_transit_hours=("transit_hours", "mean")
).reset_index().sort_values("avg_transit_hours")

display(courier_kpis.head(10))


,courier_id,total_orders,avg_transit_hours
15,C016,60,5.656324
14,C015,48,5.705341
2,C003,55,5.802268
12,C013,57,5.941408
13,C014,46,6.004909
6,C007,56,6.278507
0,C001,38,6.298041
10,C011,45,6.345630
16,C017,50,6.387511
9,C010,59,6.447815


from matplotlib import pyplot as plt
_df_0['total_orders'].plot(kind='hist', bins=20, title='total_orders')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['avg_transit_hours'].plot(kind='hist', bins=20, title='avg_transit_hours')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='total_orders', y='avg_transit_hours', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['avg_transit_hours']
  ys = series['total_orders']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('avg_transit_hours', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('avg_transit_hours')
_ = plt.ylabel('total_orders')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['avg_transit_hours']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'avg_transit_hours'}, axis=1)
              .sort_values('avg_transit_hours', ascending=True))
  xs = counted['avg_transit_hours']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('avg_transit_hours', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('avg_transit_hours')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_5['total_orders'].plot(kind='line', figsize=(8, 4), title='total_orders')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6['avg_transit_hours'].plot(kind='line', figsize=(8, 4), title='avg_transit_hours')
plt.gca().spines[['top', 'right']].set_visible(False)

In [21]:
# import pandas as pd
import plotly.express as px

# ✅ 1. Detect the correct start time column (handle renaming)
start_col = 'order_time' # Default
if 'order_time' not in df.columns and 'accept_time' in df.columns:
    start_col = 'accept_time'
    print(f"Note: Using '{start_col}' instead of 'order_time'")
elif 'order_time' not in df.columns:
    # If neither exists, try to find a date column that isn't delivery
    cols = [c for c in df.columns if 'time' in c.lower() or 'date' in c.lower()]
    cols = [c for c in cols if 'delivery' not in c.lower()]
    if cols:
        start_col = cols[0]
        print(f"Note: Auto-detected start column: '{start_col}'")
    else:
        raise KeyError("Could not find a valid start time column (e.g., 'order_time' or 'accept_time') in df.")

# ✅ 2. Force datetime parsing
df[start_col] = pd.to_datetime(df[start_col], errors="coerce")
df["delivery_time"] = pd.to_datetime(df["delivery_time"], errors="coerce")

# ✅ 3. Recalculate transit hours safely
# We calculate (Delivery - Start) in hours
df["transit_hours"] = (df["delivery_time"] - df[start_col]).dt.total_seconds() / 3600

# Drop invalid rows (where transit_hours couldn't be calculated)
df_clean = df.dropna(subset=["transit_hours"]).copy()

# ✅ 4. Create month column for aggregation
df_clean["month"] = df_clean[start_col].dt.to_period("M").dt.to_timestamp()

# ✅ 5. Monthly aggregation
monthly = df_clean.groupby("month")["transit_hours"].mean().reset_index()

print("Monthly KPI Preview:")
display(monthly.head())

# ✅ 6. Generate the Plot
fig = px.line(
    monthly,
    x="month",
    y="transit_hours",
    title="✅ Average Transit Time per Month (Corrected)"
)
fig.show()

Note: Using 'accept_time' instead of 'order_time'
Monthly KPI Preview:


,month,transit_hours
0,2024-01-01,6.397971
1,2024-02-01,7.132908
2,2024-03-01,6.474864
3,2024-04-01,6.862690
4,2024-05-01,6.704312


In [22]:

# ✅ STEP 9: Forecasting
from statsmodels.tsa.holtwinters import Holt

ts = monthly.set_index("month")["transit_hours"]

model = Holt(ts).fit()
forecast = model.forecast(3)

forecast



/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,0
2025-01-01,5.628682
2025-02-01,5.435306
2025-03-01,5.241930


In [23]:
from statsmodels.tsa.holtwinters import Holt
import pandas as pd
import plotly.express as px

# 1) Time series with monthly frequency
ts = monthly.set_index("month")["transit_hours"].asfreq("MS")

# 2) Train Holt trend model
model = Holt(ts).fit()

# 3) In-sample fitted values (for existing months)
fitted = model.fittedvalues   # same index as ts

# 4) Out-of-sample forecast (next 3 months)
forecast = model.forecast(3)  # Jan–Mar 2024

print("Forecast values:")
print(forecast)

# 5) Build combined DataFrame
actual_df = ts.reset_index()
actual_df.columns = ["month", "transit_hours"]
actual_df["type"] = "Actual"

fitted_df = fitted.reset_index()
fitted_df.columns = ["month", "transit_hours"]
fitted_df["type"] = "Fitted"

forecast_df = forecast.reset_index()
forecast_df.columns = ["month", "transit_hours"]
forecast_df["type"] = "Forecast"

plot_df = pd.concat([actual_df, fitted_df, forecast_df], ignore_index=True)

# 6) Plot
fig = px.line(
    plot_df,
    x="month",
    y="transit_hours",
    color="type",
    line_dash="type",
    markers=True,
    title="Holt Trend – Actual vs Fitted vs Forecast Transit Time"
)
fig.show()


Forecast values:
2025-01-01    5.628682
2025-02-01    5.435306
2025-03-01    5.241930
Freq: MS, dtype: float64


In [24]:
print(type(ts.index))
print(type(forecast.index))
print(ts.index[-3:])
print(forecast.index)


<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2024-10-01', '2024-11-01', '2024-12-01'], dtype='datetime64[ns]', name='month', freq='MS')
DatetimeIndex(['2025-01-01', '2025-02-01', '2025-03-01'], dtype='datetime64[ns]', freq='MS')


In [25]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Actual vs Fitted
actual = ts.values
fitted = fitted.values

# MAE
mae = mean_absolute_error(actual, fitted)

# RMSE
rmse = np.sqrt(mean_squared_error(actual, fitted))

# MAPE (in %)
mape = np.mean(np.abs((actual - fitted) / actual)) * 100

print(f"MAE  (Average Error in Hours): {mae:.4f}")
print(f"RMSE (Weighted Error):        {rmse:.4f}")
print(f"MAPE (Percentage Error):     {mape:.2f}%")


MAE  (Average Error in Hours): 0.3626
RMSE (Weighted Error):        0.5070
MAPE (Percentage Error):     5.72%


In [26]:

# ✅ STEP 10: Export Results
os.makedirs("output", exist_ok=True)

order_times.to_csv("output/order_times.csv", index=False)
courier_kpis.to_csv("output/courier_kpis.csv", index=False)
monthly.to_csv("output/monthly_kpis.csv", index=False)

print("✅ Files exported")
print(os.listdir("output"))


✅ Files exported
['courier_kpis.csv', 'monthly_kpis.csv', 'order_times.csv']


In [28]:
import pandas as pd

# 1. Identify the correct start time column (handle renaming)
if 'accept_time' in df.columns:
    start_col = 'accept_time'
    print(f"Using '{start_col}' as the start time.")
elif 'order_time' in df.columns:
    start_col = 'order_time'
    print(f"Using '{start_col}' as the start time.")
else:
    # Fallback: look for any column with 'time' or 'date' that isn't delivery
    candidates = [c for c in df.columns if ('time' in c.lower() or 'date' in c.lower()) and 'delivery' not in c.lower()]
    if candidates:
        start_col = candidates[0]
        print(f"Auto-detected start column: '{start_col}'")
    else:
        raise KeyError("Could not find a valid start time column (e.g., 'order_time' or 'accept_time') in df.")

# 2. Identify the correct ID column
id_col = 'order_uid' if 'order_uid' in df.columns else 'order_id'

# 3. Force datetime parsing
df[start_col] = pd.to_datetime(df[start_col], errors="coerce")
df["delivery_time"] = pd.to_datetime(df["delivery_time"], errors="coerce")

# 4. Recalculate transit hours
df["transit_hours"] = (df["delivery_time"] - df[start_col]).dt.total_seconds() / 3600

# 5. Group/Aggregate (if needed) or just select columns
# Note: If your data is already one-row-per-order (like order_times.csv), grouping might not be needed,
# but we'll keep it to match your original logic structure.
order_times = df.groupby(id_col)[[start_col, "delivery_time", "transit_hours"]].first().reset_index()

# 6. Create month column for trending
order_times["month"] = order_times[start_col].dt.to_period("M").dt.to_timestamp()

print(" Transit times calculated successfully.")
display(order_times.head())

Using 'accept_time' as the start time.
 Transit times calculated successfully.


,order_uid,accept_time,delivery_time,transit_hours,month
0,C001_0,2024-01-26 21:58:17,2024-01-27 05:15:05,7.280000,2024-01-01
1,C001_1,2024-01-30 18:42:20,2024-01-30 21:45:10,3.047222,2024-01-01
2,C001_10,2024-04-29 13:01:33,2024-04-29 22:13:38,9.201389,2024-04-01
3,C001_11,2024-05-02 11:35:04,2024-05-02 14:56:02,3.349444,2024-05-01
4,C001_12,2024-05-07 08:22:16,2024-05-07 10:50:16,2.466667,2024-05-01


In [29]:
courier_kpis = df.groupby("courier_id").agg(
    total_orders=("order_id", "count"),
    avg_transit_hours=("transit_hours", "mean")
).reset_index().sort_values("avg_transit_hours")

courier_kpis.tail(10)


,courier_id,total_orders,avg_transit_hours
17,C018,47,6.538534
4,C005,51,6.551176
3,C004,45,6.597099
1,C002,47,6.641962
18,C019,50,6.653894
7,C008,60,6.715449
11,C012,54,6.800941
5,C006,45,6.914852
8,C009,41,7.155549
19,C020,46,7.174662


In [31]:
import pandas as pd
import plotly.express as px

# 1. Detect the correct start time column
if 'order_time' in df.columns:
    start_col = 'order_time'
    print("Using 'order_time' column.")
elif 'accept_time' in df.columns:
    start_col = 'accept_time'
    print("Using 'accept_time' column (renamed from order_time).")
else:
    # Fallback: try to find any column with 'time' or 'date'
    candidates = [c for c in df.columns if 'time' in c.lower() or 'date' in c.lower()]
    # Exclude delivery time from candidates
    candidates = [c for c in candidates if 'delivery' not in c.lower()]

    if candidates:
        start_col = candidates[0]
        print(f"Auto-detected start column: '{start_col}'")
    else:
        raise KeyError("Could not find a valid start time column (e.g., 'order_time' or 'accept_time') in df. Please check df.columns")

# 2. Force datetime parsing
df[start_col] = pd.to_datetime(df[start_col], errors="coerce")
df["delivery_time"] = pd.to_datetime(df["delivery_time"], errors="coerce")

# 3. Recalculate transit hours
# We calculate (Delivery - Start) in hours
df["transit_hours"] = (df["delivery_time"] - df[start_col]).dt.total_seconds() / 3600

# 4. Drop invalid rows
df_clean = df.dropna(subset=["transit_hours"]).copy()

# 5. Create month column for aggregation
df_clean["month"] = df_clean[start_col].dt.to_period("M").dt.to_timestamp()

# 6. Monthly aggregation
monthly = df_clean.groupby("month")["transit_hours"].mean().reset_index()

print("Monthly KPI Preview:")
display(monthly.head())

# 7. Generate the Plot
fig = px.line(
    monthly,
    x="month",
    y="transit_hours",
    title="✅ Average Transit Time per Month (Corrected)"
)
fig.show()

Using 'accept_time' column (renamed from order_time).
Monthly KPI Preview:


,month,transit_hours
0,2024-01-01,6.397971
1,2024-02-01,7.132908
2,2024-03-01,6.474864
3,2024-04-01,6.862690
4,2024-05-01,6.704312


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
